In [6]:
# === CELL 1: IMPORTS AND SETUP ===
# Data manipulation
import pandas as pd
import numpy as np

# Parallel processing
from concurrent.futures import ProcessPoolExecutor
import warnings
import os
import time
import pickle
import psutil
import gc

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm

# Google Colab integration
from google.colab import drive

# Logging
import logging
from typing import Optional
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Install required packages
!pip install -q tqdm

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [14]:
# Import required libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from datetime import datetime
import os

# Read the data
DATA_DIR = '/content/drive/MyDrive/PNAS_Submission/'
df = pd.read_csv(DATA_DIR + 'HarvestData_STAll_Chem.csv')

# Create output directory for plots if it doesn't exist
PLOT_DIR = os.path.join(DATA_DIR, 'plots')
if not os.path.exists(PLOT_DIR):
    os.makedirs(PLOT_DIR)

# Convert date and create numeric date
df['date_adjusted'] = pd.to_datetime(df['date_adjusted'])
df['date_numeric'] = (df['date_adjusted'] - df['date_adjusted'].min()).dt.days

# Create treatment groups
df['Treatment'] = df['Site'].map(lambda x: 'Harvest' if x.startswith('H') else 'Control')

# List of optical parameters
parameters = ['fi', 'hix', 'mhix', 'bix', 'a254', 'a300', 'E2_E3',
              'S275_295', 'S350_400', 'SR']

# Calculate treatment date in numeric form
treatment_date_numeric = (pd.Timestamp('2020-09-30') - df['date_adjusted'].min()).days

# Function to remove outliers using IQR method
def remove_outliers(data, column):
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return data[(data[column] >= lower_bound) & (data[column] <= upper_bound)]

# Use subtle colors
colors = {
    'Control': '#666666',  # Darker gray
    'Harvest': '#999999'   # Lighter gray
}

# Create and save individual plots for each parameter
for param in parameters:
    # Set figure size and DPI for high-quality output
    plt.figure(figsize=(10, 6), dpi=300)

    # Plot each treatment group
    for treatment in ['Control', 'Harvest']:
        mask = df['Treatment'] == treatment
        treatment_data = df[mask].copy()

        # Remove outliers for this parameter in this treatment group
        clean_data = remove_outliers(treatment_data, param)

        # Print number of outliers removed
        n_outliers = len(treatment_data) - len(clean_data)
        if n_outliers > 0:
            print(f"{param} - {treatment}: removed {n_outliers} outliers")

        # Scatter plot with smaller points and more transparency
        plt.scatter(clean_data['date_numeric'],
                   clean_data[param],
                   color=colors[treatment],
                   label=treatment,
                   alpha=0.6,
                   s=40,
                   edgecolor='none')

    # Add vertical line for treatment date
    plt.axvline(treatment_date_numeric, color='#cccccc', linestyle='--',
                alpha=0.5, label='Treatment Date')

    # Parameter-specific labels
    param_labels = {
        'fi': 'Fluorescence Index',
        'hix': 'Humification Index',
        'mhix': 'Modified Humification Index',
        'bix': 'Biological Index',
        'a254': 'Absorbance at 254nm',
        'a300': 'Absorbance at 300nm',
        'E2_E3': 'E2:E3 Ratio',
        'S275_295': 'Spectral Slope (275-295nm)',
        'S350_400': 'Spectral Slope (350-400nm)',
        'SR': 'Slope Ratio'
    }

    # Customize the plot
    plt.title(param_labels[param], fontsize=12, pad=20)
    plt.xlabel('Days since start', fontsize=10)
    plt.ylabel(param_labels[param], fontsize=10)
    plt.grid(True, alpha=0.2)
    plt.legend(framealpha=0.8)

    # Use light gray spines
    for spine in plt.gca().spines.values():
        spine.set_color('#cccccc')

    # Tight layout
    plt.tight_layout()

    # Save the plot with high resolution
    filename = os.path.join(PLOT_DIR, f'{param}_timeseries.png')
    plt.savefig(filename, dpi=300, bbox_inches='tight', transparent=False)
    plt.close()

print(f"\nPlots have been saved to: {PLOT_DIR}")

# Print summary statistics for non-outlier data
print("\nSummary Statistics (excluding outliers):")
print("=====================================")
for param in parameters:
    print(f"\n{param}:")
    df['Period'] = df['date_adjusted'].map(lambda x: 'After' if x >= pd.Timestamp('2020-09-30') else 'Before')

    # Calculate stats on non-outlier data
    stats_list = []
    for treatment in ['Control', 'Harvest']:
        for period in ['Before', 'After']:
            subset = df[(df['Treatment'] == treatment) & (df['Period'] == period)].copy()
            clean_subset = remove_outliers(subset, param)
            stats = clean_subset[param].agg(['mean', 'std', 'count'])
            stats_list.append({'Treatment': treatment,
                             'Period': period,
                             'Stats': stats})

    # Print formatted statistics
    for stats_dict in stats_list:
        print(f"\n{stats_dict['Treatment']} - {stats_dict['Period']}:")
        print(stats_dict['Stats'])

FileNotFoundError: [Errno 2] No such file or directory: 'HarvestData_STAll_Chem.csv'

In [17]:
# Import required libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

# Read the data
DATA_DIR = '/content/drive/MyDrive/PNAS_Submission/'
df = pd.read_csv(DATA_DIR + 'HarvestData_STAll_Chem.csv')


# Convert date and create numeric date
df['date_adjusted'] = pd.to_datetime(df['date_adjusted'])
df['date_numeric'] = (df['date_adjusted'] - df['date_adjusted'].min()).dt.days

# Create treatment groups
df['Treatment'] = df['Site'].map(lambda x: 'Harvest' if x.startswith('H') else 'Control')
df['Period'] = df['date_adjusted'].map(lambda x: 'After' if x >= pd.Timestamp('2020-09-30') else 'Before')

# Function to remove outliers
def remove_outliers(data, column):
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return data[(data[column] >= lower_bound) & (data[column] <= upper_bound)]

# Get all numeric columns
numeric_cols = df.select_dtypes(include=[np.number]).columns
parameters = [col for col in numeric_cols if col not in ['date_numeric', 'order', 't', 't_mod', 'year', 'month']]

# Colors
colors = {
    'Control': '#1f77b4',  # Dark blue
    'Harvest': '#ff7f0e'   # Orange
}

# Create plots directory if it doesn't exist
if not os.path.exists('plots'):
    os.makedirs('plots')

# Create plots and collect statistics
all_stats = []

for param in parameters:
    # Create plot
    plt.figure(figsize=(10, 6), dpi=300)

    for treatment in ['Control', 'Harvest']:
        mask = df['Treatment'] == treatment
        data = df[mask].copy()
        clean_data = remove_outliers(data, param)

        plt.scatter(clean_data['date_numeric'],
                   clean_data[param],
                   color=colors[treatment],
                   label=treatment,
                   alpha=0.7,
                   s=50)

    # Add treatment date line
    treatment_date = (pd.Timestamp('2020-09-30') - df['date_adjusted'].min()).days
    plt.axvline(treatment_date, color='gray', linestyle='--', alpha=0.5, label='Treatment Date')

    plt.title(param)
    plt.xlabel('Days since start')
    plt.ylabel(param)
    plt.grid(True, alpha=0.2)
    plt.legend()
    plt.tight_layout()

    # Save plot
    plt.savefig(DATA_DIR + f'plots/{param.replace(".", "_")}.png', bbox_inches='tight')
    plt.close()

    # Calculate statistics
    for treatment in ['Control', 'Harvest']:
        for period in ['Before', 'After']:
            subset = df[(df['Treatment'] == treatment) & (df['Period'] == period)]
            clean_subset = remove_outliers(subset, param)

            stats = {
                'Parameter': param,
                'Treatment': treatment,
                'Period': period,
                'N': len(clean_subset),
                'Mean': clean_subset[param].mean(),
                'SD': clean_subset[param].std(),
                'Min': clean_subset[param].min(),
                'Max': clean_subset[param].max()
            }
            all_stats.append(stats)

# Save statistics to CSV
stats_df = pd.DataFrame(all_stats)
stats_df = stats_df.round(4)  # Round to 4 decimal places
stats_df.to_csv(DATA_DIR +'parameter_statistics.csv', index=False)

print("Plots saved to 'plots' directory")
print("Statistics saved to 'parameter_statistics.csv'")

Plots saved to 'plots' directory
Statistics saved to 'parameter_statistics.csv'
